In [22]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities["MLB"] = cities["MLB"].str.replace(r"\[.+\]", "", regex=True)
    cities["MLB"] = cities["MLB"].replace({
                                           "Cubs White Sox": "Cubs,White Sox", 
                                           "Dodgers Angels": "Dodgers,Angels", 
                                           "Giants Athletics": "Giants,Athletics", 
                                           "Yankees Mets": "Yankees,Mets",
                                           })
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(','))
    cities = cities.explode("MLB")
    cities

    infocities = cities.iloc[:, [0,1,3]].copy()

    infocities.rename(columns={0: 'Metropolitan area', 1: 'Population'}, inplace=True)
    infocities.columns = ['Metropolitan area', 'Population', 'MLB']
    infocities = infocities[infocities['MLB'].str.contains(r'[a-zA-Z]')]
    infocities["MLB"]

    info_league = mlb_df[mlb_df['year'] == 2018][['team', 'W', 'L']]
    info_league['team'] = info_league['team'].str.replace('*', '')
    info_league = info_league[~info_league['team'].str.contains('Division')]
    info_league['team'] = info_league["team"].str.replace(r"\(.+\)", "", regex=True)
    info_league

    teams_shorts = infocities["MLB"].str.strip()
    teams_complete = info_league["team"].str.strip()

    final_list = []

    for short_name in teams_shorts:

        idx = teams_complete[teams_complete.str.contains(short_name)].index

        if not idx.empty:
            index = idx[0]

            team_complete_row = info_league.loc[index]

            infocities_row = infocities[infocities['MLB'] == short_name].iloc[0]

            combined_row = pd.concat([team_complete_row, infocities_row])

            final_list.append(combined_row)

    final_df = pd.DataFrame(final_list)
    final_df

    result_df = final_df.copy()

    result_df['W'] = pd.to_numeric(result_df['W'])
    result_df['L'] = pd.to_numeric(result_df['L'])

    result_df['Win-Loss Ratio'] = result_df['W'] / (result_df['W'] + result_df['L'])
    result_df = result_df.drop(["team", "MLB"], axis=1)

    # print(len(result_df.drop_duplicates(subset=["Metropolitan area"]))) # Ya son 28 valores
    result_df

    df_unique = result_df.drop_duplicates(subset=['Metropolitan area'])
    mean_ratio = result_df.groupby('Metropolitan area')['Win-Loss Ratio'].mean().reset_index()
    mean_ratio

    population_data = result_df.groupby('Metropolitan area')['Population'].first().reset_index()
    population_data

    mean_ratio_with_population = pd.merge(mean_ratio, population_data, on='Metropolitan area', how='left')
    mean_ratio_with_population

    mean_ratio_with_population['Population'] = mean_ratio_with_population['Population'].astype(float)

    population_by_region = mean_ratio_with_population['Population']
    win_loss_by_region = mean_ratio_with_population['Win-Loss Ratio'] 

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    final = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return final

mlb_correlation()



0.15027698302669307